In [13]:
import pandas as pd 
import numpy as np 
import seaborn as sns 
sns.set(style='dark', palette='muted', color_codes=True)
import matplotlib.pyplot as plt 
plt.style.use('ggplot')
import plotly.express as px 

In [14]:
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder,OneHotEncoder,StandardScaler,MinMaxScaler 
from sklearn.compose import ColumnTransformer 
from sklearn.metrics import r2_score,mean_squared_error,silhouette_score,accuracy_score,confusion_matrix 
from sklearn.linear_model import LinearRegression,LogisticRegression 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.cluster import KMeans 
from sklearn.decomposition import PCA 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [15]:
data = pd.read_csv(r'c:\Github\Sol\the_data\data-lab-5-hcv.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  615 non-null    int64  
 1   Category    615 non-null    object 
 2   Age         615 non-null    int64  
 3   Sex         615 non-null    object 
 4   ALB         614 non-null    float64
 5   ALP         597 non-null    float64
 6   ALT         614 non-null    float64
 7   AST         615 non-null    float64
 8   BIL         615 non-null    float64
 9   CHE         615 non-null    float64
 10  CHOL        605 non-null    float64
 11  CREA        615 non-null    float64
 12  GGT         615 non-null    float64
 13  PROT        614 non-null    float64
dtypes: float64(10), int64(2), object(2)
memory usage: 67.4+ KB


In [16]:
mis_val = data.isna().sum() * 100/len(data)

mis_val.sort_values(ascending=False)

ALP           2.926829
CHOL          1.626016
ALB           0.162602
ALT           0.162602
PROT          0.162602
Unnamed: 0    0.000000
Category      0.000000
Age           0.000000
Sex           0.000000
AST           0.000000
BIL           0.000000
CHE           0.000000
CREA          0.000000
GGT           0.000000
dtype: float64

In [17]:
data.duplicated().sum()
data.drop_duplicates(inplace=True, keep='last', subset=None, ignore_index=False)
data.duplicated().sum()

0

In [18]:
mean_imp = SimpleImputer(missing_values=np.NaN, strategy='mean')

data[['ALP', 'CHOL', 'ALB', 'ALT', 'PROT']] = mean_imp.fit_transform(data[['ALP', 'CHOL', 'ALB', 'ALT', 'PROT']] )

In [19]:
mis_val = data.isna().sum() * 100/len(data)

mis_val.sort_values(ascending=False)

Unnamed: 0    0.0
Category      0.0
Age           0.0
Sex           0.0
ALB           0.0
ALP           0.0
ALT           0.0
AST           0.0
BIL           0.0
CHE           0.0
CHOL          0.0
CREA          0.0
GGT           0.0
PROT          0.0
dtype: float64

In [20]:
le = LabelEncoder()

y = data['Category']

y_true = le.fit_transform(y)

data = pd.get_dummies(data, dtype=float)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 615 entries, 0 to 614
Data columns (total 19 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Unnamed: 0                       615 non-null    int64  
 1   Age                              615 non-null    int64  
 2   ALB                              615 non-null    float64
 3   ALP                              615 non-null    float64
 4   ALT                              615 non-null    float64
 5   AST                              615 non-null    float64
 6   BIL                              615 non-null    float64
 7   CHE                              615 non-null    float64
 8   CHOL                             615 non-null    float64
 9   CREA                             615 non-null    float64
 10  GGT                              615 non-null    float64
 11  PROT                             615 non-null    float64
 12  Category_0=Blood Donor